In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

from keras import backend as K
import keras
import os, re, sys
import pandas as pd
from keras.utils import to_categorical
import ast
from sklearn.utils import class_weight
from keras.preprocessing import sequence
from keras.models import load_model
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
import keras
from sklearn.preprocessing import normalize

from sklearn.metrics import *

'''
Compatible with tensorflow backend
gamma entre más alto más tolerante (queremos un gamma chico pero no demasiado! 0.25 - 5)
alpha pesos por clase (weights)
'''
def focal_loss(gamma=2., weights=1):   #weights np.asarray()
    weights= K.variable(weights)
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(),1)
        y_pred = K.clip(y_pred,K.epsilon(),1)
        return - K.sum(weights* K.pow(1. - y_pred, gamma)* y_true * K.log(y_pred), axis=-1) 
    return focal_loss_fixed

keras.losses.focal_loss=keras.losses.MSE
keras.losses.focal_loss_fixed=keras.losses.MSE

Using TensorFlow backend.


In [3]:
from keras.models import load_model
cnn1_f = load_model('.h5')
cnn2_f = load_model('Modelos_alt/cnn2_new_unbalanced_focal.h5')
rnn2_f1 = load_model('Modelos_alt/rnn2_new_unbalanced_focal_10.h5')
rnn3_c = load_model('Modelos_alt/rnn3_new_unbalanced.h5')
rnn3_f1 = load_model('Modelos_alt/rnn3_new_unbalanced_focal_10.h5')

OSError: Unable to open file (unable to open file: name = 'Modelos_alt/cnn1_new_unbalanced_focal.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
list_models=['cnn1_f', 'cnn2_f', 'rnn2_f1', 'rnn3_c', 'rnn3_f1']
index_models=np.arange(5)
dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
modelos=[cnn1_f, cnn2_f, rnn2_f1, rnn3_c, rnn3_f1]
ind=np.arange(5)
dict_trainingModel=dict((key, value) for (key, value) in zip(ind,modelos))

In [ ]:
predicciones_all=[]
X_train=np.load("../x_train_models.npy")
X_train_rnn=np.load("../x_train_rnn_models.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='rnn':
        predicciones_all.append(dict_trainingModel[i].predict(X_train, batch_size=bs))
    else:
        predicciones_all.append(dict_trainingModel[i].predict(X_train_rnn, batch_size=bs))
        
predicciones_all_val=[]
X_val=np.load("../x_val_models.npy")
X_val_rnn=np.load("../x_val_rnn_models.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones val del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='rnn':
        predicciones_all_val.append(dict_trainingModel[i].predict(X_val, batch_size=bs))
    else:
        predicciones_all_val.append(dict_trainingModel[i].predict(X_val_rnn, batch_size=bs))
        
predicciones_all_test=[]
X_test=np.load("x_test_models.npy")
X_test_rnn=np.load("x_test_rnn_models.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones val del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='rnn':
        predicciones_all_test.append(dict_trainingModel[i].predict(X_test, batch_size=bs))
    else:
        predicciones_all_test.append(dict_trainingModel[i].predict(X_test_rnn, batch_size=bs))

In [ ]:
def predecir_modelos(X_train,X_train_rnn,bs,predichos_all):
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  #vectores de largo 60
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        if final[i]%4==0:
            predichos_new.append(0)
        if final[i]%4==1:
            predichos_new.append(1)
        if final[i]%4==2:
            predichos_new.append(2)
        if final[i]%4==3:
            predichos_new.append(3)
    return predichos_new,final,confianzas    

def predecir_modelos_norm(X_train,X_train_rnn,bs,predichos_all):
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:  #vectores de largo 60  
        aux=np.ones(4)
        #print ("aux en norm",aux)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            #print ("sub en norm",sub)
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas#, predichos_new_var
    
def predecir_modelos_average(X_train,X_train_rnn,bs,predichos_all):
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:  #vectores de largo 60  
        aux=np.ones(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/15.0
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas#, predichos_new_var

In [ ]:
predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
print (predichos_all_val.shape)
new_y_val=np.load("y_val_models.npy")
len(new_y_val)

In [ ]:
batchSize=32
trainPredicho, maxPredicho, confianzas = predecir_modelos(X_test,X_test_rnn, batchSize,predichos_all_test)
acc_comite= accuracy_score(new_y_test, trainPredicho)  
print("Accuracy test del comité:",acc_comite)

f1=f1_score(new_y_test, trainPredicho, average='weighted')  # labels=np.unique(trainPredict)
print("F1-score weighted test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='micro')  # labels=np.unique(trainPredict)
print("F1-score micro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='macro')  # labels=np.unique(trainPredict)
print("F1-score macro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average=None)  # labels=np.unique(trainPredict)
print("F1-score macro test del comité desagregado:",f1)

In [ ]:
batchSize=32
trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(X_test,X_test_rnn, batchSize,predichos_all_test)
acc_comite= accuracy_score(new_y_test, trainPredicho)  
print("Accuracy test del comité:",acc_comite)

f1=f1_score(new_y_test, trainPredicho, average='weighted')  # labels=np.unique(trainPredict)
print("F1-score weighted test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='micro')  # labels=np.unique(trainPredict)
print("F1-score micro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='macro')  # labels=np.unique(trainPredict)
print("F1-score macro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average=None)  # labels=np.unique(trainPredict)
print("F1-score macro test del comité desagregado:",f1)

In [ ]:
batchSize=32
trainPredicho, maxPredicho, confianzas = predecir_modelos_average(X_test,X_test_rnn, batchSize,predichos_all_test)
acc_comite= accuracy_score(new_y_test, trainPredicho)  
print("Accuracy test del comité:",acc_comite)

f1=f1_score(new_y_test, trainPredicho, average='weighted')  # labels=np.unique(trainPredict)
print("F1-score weighted test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='micro')  # labels=np.unique(trainPredict)
print("F1-score micro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average='macro')  # labels=np.unique(trainPredict)
print("F1-score macro test del comité:",f1)
f1=f1_score(new_y_test, trainPredicho, average=None)  # labels=np.unique(trainPredict)
print("F1-score macro test del comité desagregado:",f1)


### Resultados por Baseline

In [ ]:
X_test=np.load("x_test_models.npy")
X_test_rnn=np.load("x_test_rnn_models.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones val del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='rnn':
        predicho=dict_trainingModel[i].predict(X_test, batch_size=bs)
    else:
        predicho=dict_trainingModel[i].predict(X_test_rnn, batch_size=bs)
    
    predicho=[np.argmax(pred) for pred in predicho]
    acc_tr= accuracy_score(new_y_test, predicho)  
    f1_ma=f1_score(new_y_test, predicho, average='macro')
    f1_no=f1_score(new_y_test, predicho, average=None)

    print ("Accuracy sobre entrenamiento gru FOCAL:",acc_tr)  
    print ("F1-score macro sobre entrenamiento gru FOCAL:",f1_ma)
    print ("F1-score none sobre entrenamiento gru FOCAL:",f1_no)